# PDF sayfa sayısı 

In [26]:
from pypdf import PdfReader, PdfWriter

# Dosya yolları
input_path = "/Users/kemalgunay/Downloads/Example.pdf"
output_path = "first_two_pages.pdf"

# PDF'i oku
reader = PdfReader(input_path)
writer = PdfWriter()

# İlk iki sayfayı ekle
for page_num in range(2):  # 0 ve 1. sayfalar
    writer.add_page(reader.pages[page_num])

# Yeni PDF olarak kaydet
with open(output_path, "wb") as f:
    writer.write(f)

print(f"✅ First two pages saved to: {output_path}")


✅ First two pages saved to: first_two_pages.pdf


# GEMINI ALL BRANCHES

In [37]:
import pandas as pd
import csv
from pypdf import PdfReader
import json
from google import genai
import os

# ----------------------------------------------------------------------
# 1. Yapılandırma
# ----------------------------------------------------------------------

# PDF Dosya Yolu
PDF_PATH = "/Users/kemalgunay/first_two_pages.pdf"

# CSV Dosya Yolu (Masaüstü)
CSV_PATH = "/Users/kemalgunay/Desktop/extracted_company_data.csv"

# Gemini API Anahtarı (Ortam değişkeninden alınır)
os.environ["GEMINI_API_KEY"] = "Your-Gemini-Api" 
client = genai.Client()

# ----------------------------------------------------------------------
# 2. PDF'ten Ham Metin Çıkarma
# ----------------------------------------------------------------------
def extract_raw_text(pdf_path):
    reader = PdfReader(pdf_path)
    full_text = ""
    print(f"Toplam sayfa sayısı: {len(reader.pages)}")
    
    for i, page in enumerate(reader.pages):
        page_text = page.extract_text() or ""
        full_text += page_text
    
    print(f"Ham metin uzunluğu: {len(full_text)} karakter")
    return full_text

# ----------------------------------------------------------------------
# 3. LLM ile JSON Verisi Çıkarma
# ----------------------------------------------------------------------
def extract_with_llm(text_chunk, client):
    if not text_chunk.strip():
        return []

    PROMPT = f"""
    Aşağıdaki metin, üç sütunlu bir iş rehberi formatındadır. 
    Metindeki şirket bilgilerini analiz edin ve her bir şirketi, 
    belirtilen tüm değişkenlerle birlikte bir Python listesi içine yerleştirilmiş 
    JSON nesneleri olarak çıkarın. 
    Sayfa sonlarında kesilen veya bir sonraki sütuna taşan bilgileri birleştirin.

    İstenen alanlar:
    - CompanyName
    - Address
    - PhoneNumber_Main
    - Email
    - Website
    - CoreActivity
    - Branches (Liste formatında, her bir şube için: {{"BranchLocation", "BranchPhoneNumber"}})
    
    Çıktı sadece ve sadece JSON listesi olmalıdır.

    Metin:
    ---
    {text_chunk}
    ---
    """
    
    try:
        response = client.models.generate_content(
            model='gemini-2.5-flash',
            contents=PROMPT,
            config={"response_mime_type": "application/json"}
        )
        return json.loads(response.text)
    except Exception as e:
        print(f"LLM çıkarma hatası: {e}")
        return []

# ----------------------------------------------------------------------
# 4. Ana Akış
# ----------------------------------------------------------------------
def main_scraper():
    print(f"PDF'ten ham metin çıkarılıyor: {PDF_PATH}")
    full_raw_text = extract_raw_text(PDF_PATH)
    
    if not full_raw_text:
        print("PDF'den metin alınamadı. İşlem sonlandırılıyor.")
        return

    # Metni parçala
    chunk_size = 2000
    text_chunks = [full_raw_text[i:i + chunk_size] for i in range(0, len(full_raw_text), chunk_size)]
    print(f"Metin {len(text_chunks)} parçaya bölündü.")

    # CSV başlıkları
    fieldnames = [
        "CompanyName", 
        "Address", 
        "PhoneNumber_Main", 
        "Email", 
        "Website", 
        "CoreActivity", 
    ]
    
    # 1'den 30'a kadar şube sütunlarını ekle
    MAX_BRANCHES = 30
    for i in range(1, MAX_BRANCHES + 1):
        fieldnames.append(f"BranchLocation_{i}")
        fieldnames.append(f"BranchPhoneNumber_{i}")
    total_companies_extracted = 0

    with open(CSV_PATH, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames, extrasaction='ignore')
        writer.writeheader()

        for i, chunk in enumerate(text_chunks):
            print(f"--- Parça {i+1}/{len(text_chunks)} Gemini ile işleniyor...")
            company_data_list = extract_with_llm(chunk, client=client)
            
            if not company_data_list:
                print(f"Parça {i+1}: Veri alınamadı, sonraki parçaya geçiliyor.")
                continue
            
            for company in company_data_list:
                csv_row = {
                    "CompanyName": company.get("CompanyName", ""),
                    "Address": company.get("Address", ""),
                    "PhoneNumber_Main": company.get("PhoneNumber_Main", ""),
                    "Email": company.get("Email", ""),
                    "Website": company.get("Website", ""),
                    "CoreActivity": company.get("CoreActivity", ""),
                }

                # Şubeleri CSV'ye ekle
                branches = company.get("Branches", [])
                for j, branch in enumerate(branches):
                    branch_loc_key = f"BranchLocation_{j+1}"
                    branch_phone_key = f"BranchPhoneNumber_{j+1}"
                    if branch_loc_key in fieldnames:
                        csv_row[branch_loc_key] = branch.get("BranchLocation", "")
                        csv_row[branch_phone_key] = branch.get("BranchPhoneNumber", "")
                
                writer.writerow(csv_row)
                total_companies_extracted += 1

    print("\n---------------------------------------------------")
    print(f"İşlem tamamlandı! Toplam {total_companies_extracted} şirket çıkarıldı.")
    print(f"Veriler '{CSV_PATH}' dosyasına kaydedildi.")
    print("---------------------------------------------------")

# Çalıştırma
if __name__ == '__main__':
    main_scraper()


PDF'ten ham metin çıkarılıyor: /Users/kemalgunay/first_two_pages.pdf
Toplam sayfa sayısı: 2
Ham metin uzunluğu: 6056 karakter
Metin 4 parçaya bölündü.
--- Parça 1/4 Gemini ile işleniyor...
--- Parça 2/4 Gemini ile işleniyor...
--- Parça 3/4 Gemini ile işleniyor...
--- Parça 4/4 Gemini ile işleniyor...

---------------------------------------------------
İşlem tamamlandı! Toplam 24 şirket çıkarıldı.
Veriler '/Users/kemalgunay/Desktop/extracted_company_data.csv' dosyasına kaydedildi.
---------------------------------------------------
